In [20]:
import tensorflow as tf
from attrdict import AttrDict
tf.__version__

'1.8.0'

读取文件

In [14]:
import re
import tarfile

class ReviewReader:
    
    TOKEN_REGEX = re.compile(r'[A-Za-z]+|[!?.:,()]')
    
    def __init__(self):
        filepath="aclImdb_v1.tar.gz"
        self.train_data = []
        self.test_data = []
        
        with tarfile.open(filepath) as archive:
            for filename in archive.getnames():
                if filename.startswith('aclImdb/train/pos/'):
                    self.train_data.append((self._read(archive, filename), True))
                elif filename.startswith('aclImdb/train/neg/'):
                    self.train_data.append((self._read(archive, filename), False))
                elif filename.startswith('aclImdb/test/pos/'):
                    self.test_data.append((self._read(archive, filename), True))
                elif filename.startswith('aclImdb/test/neg/'):
                    self.test_data.append((self._read(archive, filename), False))
    
    def _read(self, archive, filename):
        with archive.extractfile(filename) as file_:
            data = file_.read().decode('utf-8')
            data = type(self).TOKEN_REGEX.findall(data)
            data = [x.lower() for x in data]
            return data
    
    

    

In [15]:
rr = ReviewReader()
test_data = rr.test_data
train_data = rr.train_data
print("test.shape:", len(test_data))
print("train.shape:", len(train_data))

test.shape: 25000
train.shape: 25000


In [28]:
graph = tf.Graph()

length = 2700
embedding_dimensions = 300

params = AttrDict(
        rnn_cell = tf.nn.rnn_cell.GRUCell,
        rnn_hidden = 300,
        optimizer = tf.train.RMSPropOptimizer(0.002),
        batch_size = 20,
    )

with graph.as_default():
    data = tf.placeholder(tf.float32, [None, length, embedding_dimensions], name="data")
    target = tf.placeholder(tf.float32, [None, 2], name = "target")
    used = tf.sign(tf.reduce_max(tf.abs(data), reduction_indices = 2))
    temp_length = tf.reduce_sum(used, reduction_indices = 1)
    seq_length = tf.cast(temp_length, tf.int32)

    output, _ = tf.nn.dynamic_rnn(params.rnn_cell(params.rnn_hidden),
        data,
        dtype = tf.float32,
        sequence_length = seq_length
    )
    

In [29]:
writer = tf.summary.FileWriter("./graph_temp", graph)
writer.close()